In [ ]:
# Import required libraries 
import cv2 
import numpy as np 
import dlib


# Connects to your computer's default camera 
cap = cv2.VideoCapture(0) 


# Detect the coordinates 
detector = dlib.get_frontal_face_detector() 


# Capture frames continuously 
while True: 

	# Capture frame-by-frame 
	ret, frame = cap.read() 
	frame = cv2.flip(frame, 1) 

	# RGB to grayscale 
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
	faces = detector(gray) 

	# Iterator to count faces 
	i = 0
	for face in faces: 

		# Get the coordinates of faces 
		x, y = face.left(), face.top() 
		x1, y1 = face.right(), face.bottom() 
		cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2) 

		# Increment iterator for each face in faces 
		i = i+1

		# Display the box and faces 
		cv2.putText(frame, 'face num'+str(i), (x-10, y-10), 
					cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
		print(face, i) 

	# Display the resulting frame 
	cv2.imshow('frame', frame) 

	# This command let's us quit with the "q" button on a keyboard. 
	if cv2.waitKey(1) & 0xFF == ord('q'): 
		break


# Release the capture and destroy the windows 
cap.release() 
cv2.destroyAllWindows() 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv('Tesla.csv')
df.head()


In [ ]:
df.shape


In [ ]:
df.describe()


In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df['Close'])
plt.title('Tesla Close price.', fontsize=15)
plt.ylabel('Price in dollars.')
plt.show()


In [ ]:
df.head()


In [ ]:
df = df.drop(['Adj Close'], axis=1)


In [ ]:
df.isnull().sum()


In [ ]:
features = ['Open', 'High', 'Low', 'Close', 'Volume']

plt.subplots(figsize=(20,10))

for i, col in enumerate(features):
    plt.subplot(2,3,i+1)
    sb.distplot(df[col])
plt.show()


In [ ]:
plt.subplots(figsize=(20,10))
for i, col in enumerate(features):
    plt.subplot(2,3,i+1)
    sb.boxplot(df[col])
plt.show()


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

In [ ]:
df.head()

In [ ]:
df['is_quarter_end'] = np.where(df['month']%3==0,1,0)
df.head()


In [ ]:
data_grouped = df.groupby('year').mean()
plt.subplots(figsize=(20,10))

for i, col in enumerate(['Open', 'High', 'Low', 'Close']):
    plt.subplot(2,2,i+1)
    data_grouped[col].plot.bar()
plt.show()


In [ ]:
df.groupby('is_quarter_end').mean()


In [ ]:
df['open-close'] = df['Open'] - df['Close']
df['low-high'] = df['Low'] - df['High']
df['target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)


In [ ]:
plt.pie(df['target'].value_counts().values, 
		labels=[0, 1], autopct='%1.1f%%')
plt.show()


In [ ]:
plt.figure(figsize=(10, 10))

# As our concern is with the highly
# correlated features only so, we will visualize
# our heatmap as per that criteria only.
sb.heatmap(df.corr() > 0.9, annot=True, cbar=False)
plt.show()


In [ ]:
features = df[['open-close', 'low-high', 'is_quarter_end']]
target = df['target']

scaler = StandardScaler()
features = scaler.fit_transform(features)

X_train, X_valid, Y_train, Y_valid = train_test_split(
	features, target, test_size=0.1, random_state=2022)
print(X_train.shape, X_valid.shape)


In [ ]:
models = [LogisticRegression(), SVC(
kernel='poly', probability=True), XGBClassifier()]

for i in range(3):
    models[i].fit(X_train, Y_train)

    print(f'{models[i]} : ')
    print('Training Accuracy : ', metrics.roc_auc_score(
	    Y_train, models[i].predict_proba(X_train)[:,1]))
    print('Validation Accuracy : ', metrics.roc_auc_score(
	    Y_valid, models[i].predict_proba(X_valid)[:,1]))
    print()
